<a href="https://colab.research.google.com/github/ryannov4/Capstone-Project/blob/main/capstone_project_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/ibm-granite-community/utils \
    "langchain_community<0.3.0" \
    replicate

  Cloning https://github.com/ibm-granite-community/utils to /tmp/pip-req-build-qdk4t2ep
  Running command git clone --filter=blob:none --quiet https://github.com/ibm-granite-community/utils /tmp/pip-req-build-qdk4t2ep
  Resolved https://github.com/ibm-granite-community/utils to commit 97732c007b2768d5c61dc61169743373043252ec
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from ibm_granite_community.notebook_utils import get_env_var
from langchain_community.llms import Replicate

model = Replicate(
    model="ibm-granite/granite-3.3-8b-instruct",
    replicate_api_token=get_env_var('REPLICATE_API_TOKEN'),
    model_kwargs={"max_tokens":4080, "temperature":0.2},
)

In [ ]:
examples_dashboard = [
    {
        "question": "Create the Dashboard page for the Personal Expense Tracker app with summary info (total income, total expense, final balance), interactive Pie Chart (income vs expense), Bar Chart (expense per category), Line Chart (balance over time). Add navigation buttons to Transaction Management, Activity History, and Analytics pages. Include toggle for Light/Dark mode.",
        "context": "Data is stored in localStorage. Charts use Chart.js. Page updates dynamically on data changes. Light/Dark mode preference saved in localStorage.",
        "output": """
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8" />
<meta name="viewport" content="width=device-width, initial-scale=1" />
<title>Expense Tracker Dashboard</title>
<script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
<style>
  :root {
    --bg-light: #f9f9f9;
    --text-light: #333;
    --card-light: #fff;
    --bg-dark: #121212;
    --text-dark: #eee;
    --card-dark: #1e1e1e;
    --primary: #2196f3;
    --success: #4caf50;
    --danger: #f44336;
  }
  body {
    margin: 0;
    font-family: Arial, sans-serif;
    background: var(--bg-light);
    color: var(--text-light);
    transition: all 0.3s ease;
  }
  body.dark {
    background: var(--bg-dark);
    color: var(--text-dark);
  }
  .container {
    max-width: 1000px;
    margin: auto;
    padding: 20px;
  }
  h1 {
    text-align: center;
    margin-bottom: 20px;
  }
  .summary {
    display: flex;
    flex-wrap: wrap;
    gap: 15px;
    margin-bottom: 20px;
  }
  .summary-card {
    flex: 1;
    min-width: 200px;
    padding: 15px;
    border-radius: 8px;
    background: var(--card-light);
    text-align: center;
    box-shadow: 0 2px 5px rgba(0,0,0,0.1);
    transition: all 0.3s ease;
  }
  body.dark .summary-card {
    background: var(--card-dark);
    box-shadow: 0 2px 5px rgba(0,0,0,0.6);
  }
  .summary-card h3 {
    margin: 0 0 10px;
  }
  .summary-card p {
    font-size: 1.2em;
    font-weight: bold;
  }
  .chart-container {
    background: var(--card-light);
    padding: 15px;
    margin-bottom: 20px;
    border-radius: 8px;
    box-shadow: 0 2px 5px rgba(0,0,0,0.1);
    transition: all 0.3s ease;
  }
  body.dark .chart-container {
    background: var(--card-dark);
    box-shadow: 0 2px 5px rgba(0,0,0,0.6);
  }
  .nav-buttons {
    text-align: center;
    margin-top: 10px;
  }
  .nav-buttons button {
    margin: 5px;
    padding: 10px 15px;
    border: none;
    background: var(--primary);
    color: white;
    border-radius: 5px;
    cursor: pointer;
    transition: background 0.3s;
  }
  .nav-buttons button:hover {
    background: #1976d2;
  }
  #toggleThemeBtn {
    background: var(--success);
  }
  #toggleThemeBtn:hover {
    background: #388e3c;
  }
</style>
</head>
<body>
<div class="container">
  <h1>Personal Expense Tracker - Dashboard</h1>

  <div class="summary">
    <div class="summary-card">
      <h3>Total Income</h3>
      <p id="total-income">Rp 0</p>
    </div>
    <div class="summary-card">
      <h3>Total Expense</h3>
      <p id="total-expense">Rp 0</p>
    </div>
    <div class="summary-card">
      <h3>Balance</h3>
      <p id="balance">Rp 0</p>
    </div>
  </div>

  <div class="chart-container">
    <canvas id="pieChart"></canvas>
  </div>
  <div class="chart-container">
    <canvas id="barChart"></canvas>
  </div>
  <div class="chart-container">
    <canvas id="lineChart"></canvas>
  </div>

  <div class="nav-buttons">
    <button onclick="navigateTo('transactions')">Manage Transactions</button>
    <button onclick="navigateTo('history')">Activity History</button>
    <button onclick="navigateTo('analytics')">Analytics Detail</button>
    <button id="toggleThemeBtn">Toggle Light/Dark Mode</button>
  </div>
</div>

<script>
const storageKey = 'transactions';

function getTransactions() {
  const data = localStorage.getItem(storageKey);
  return data ? JSON.parse(data) : [];
}

function calculateSummary(transactions) {
  let income = 0;
  let expense = 0;
  transactions.forEach(t => {
    if(t.category === 'Pemasukan') income += parseFloat(t.amount);
    else expense += parseFloat(t.amount);
  });
  return { income, expense, balance: income - expense };
}

function groupExpenseByCategory(transactions) {
  const categoryTotals = {};
  transactions.forEach(t => {
    if(t.category === 'Pengeluaran') {
      const cat = t.description || "Lainnya";
      categoryTotals[cat] = (categoryTotals[cat] || 0) + parseFloat(t.amount);
    }
  });
  return categoryTotals;
}

function getBalanceOverTime(transactions) {
  let runningBalance = 0;
  const grouped = {};
  transactions.sort((a,b) => new Date(a.date) - new Date(b.date));
  transactions.forEach(t => {
    runningBalance += (t.category === 'Pemasukan' ? 1 : -1) * parseFloat(t.amount);
    grouped[t.date] = runningBalance;
  });
  return grouped;
}

function renderSummary() {
  const transactions = getTransactions();
  const { income, expense, balance } = calculateSummary(transactions);
  document.getElementById('total-income').textContent = 'Rp ' + income.toLocaleString();
  document.getElementById('total-expense').textContent = 'Rp ' + expense.toLocaleString();
  document.getElementById('balance').textContent = 'Rp ' + balance.toLocaleString();
  return transactions;
}

let pieChart, barChart, lineChart;

function renderCharts(transactions) {
  const ctxPie = document.getElementById('pieChart').getContext('2d');
  const ctxBar = document.getElementById('barChart').getContext('2d');
  const ctxLine = document.getElementById('lineChart').getContext('2d');

  const summary = calculateSummary(transactions);
  const expenseByCategory = groupExpenseByCategory(transactions);
  const balanceOverTime = getBalanceOverTime(transactions);

  if(pieChart) pieChart.destroy();
  pieChart = new Chart(ctxPie, {
    type: 'pie',
    data: {
      labels: ['Income', 'Expense'],
      datasets: [{
        data: [summary.income, summary.expense],
        backgroundColor: [varColor('--success'), varColor('--danger')]
      }]
    }
  });

  if(barChart) barChart.destroy();
  barChart = new Chart(ctxBar, {
    type: 'bar',
    data: {
      labels: Object.keys(expenseByCategory),
      datasets: [{
        label: 'Expense by Category',
        data: Object.values(expenseByCategory),
        backgroundColor: varColor('--danger')
      }]
    },
    options: {
      responsive: true,
      scales: { y: { beginAtZero: true } }
    }
  });

  if(lineChart) lineChart.destroy();
  lineChart = new Chart(ctxLine, {
    type: 'line',
    data: {
      labels: Object.keys(balanceOverTime),
      datasets: [{
        label: 'Balance Over Time',
        data: Object.values(balanceOverTime),
        borderColor: varColor('--primary'),
        fill: false
      }]
    },
    options: {
      responsive: true,
      scales: { y: { beginAtZero: true } }
    }
  });
}

function varColor(name) {
  return getComputedStyle(document.documentElement).getPropertyValue(name).trim();
}

function navigateTo(page) {
  alert('Navigate to ' + page + ' page (functionality to be implemented)');
}

function loadTheme() {
  const theme = localStorage.getItem('theme') || 'light';
  document.body.className = theme;
}

function toggleTheme() {
  let current = document.body.className;
  const newTheme = current === 'light' ? 'dark' : 'light';
  document.body.className = newTheme;
  localStorage.setItem('theme', newTheme);
}

document.getElementById('toggleThemeBtn').addEventListener('click', toggleTheme);

window.onload = () => {
  loadTheme();
  const transactions = renderSummary();
  renderCharts(transactions);
};
</script>
</body>
</html>
"""
    }
]


In [ ]:
def fewshot_prompt_dashboard(context, question, examples):
    formatted_examples = "\n\n".join(
        f"""
Example {i+1}:
User Question: {ex['question']}
Context: {ex['context']}
Model Output:
{ex['output']}
        """ for i, ex in enumerate(examples)
    )

    prompt = f"""
You are an expert full-stack developer with 15 years of experience.
Your task is to generate a high-quality HTML, CSS, and JavaScript code for the Dashboard page
of a Personal Expense Tracker app, based on the context and user question below.

Requirements:
- Must include complete HTML structure, inline or internal CSS, and JavaScript.
- Use responsive design (mobile-friendly).
- Implement all features described in the question.
- Save and retrieve data from localStorage as described.
- Charts must use Chart.js.
- Light/Dark mode preference saved in localStorage.
- Output only the complete code for this page. Do not include explanations or comments outside of the code.

Here are examples of similar tasks you have completed before:
{formatted_examples}

Now generate the Dashboard page code:

Context:
{context}

User Question:
{question}

Output:
"""
    return prompt

def get_answer_dashboard(context, question, examples):
    prompt = fewshot_prompt_dashboard(context, question, examples)
    result = model.invoke(prompt)
    return result

# Example usage
context = "Data is stored in localStorage. Charts use Chart.js. Page updates dynamically on data changes. Light/Dark mode preference saved in localStorage."
question = "Create the Dashboard page for the Personal Expense Tracker app with summary info (total income, total expense, final balance), interactive Pie Chart (income vs expense), Bar Chart (expense per category), Line Chart (balance over time). Add navigation buttons to Transaction Management, Activity History, and Analytics pages. Include toggle for Light/Dark mode."
result_dashboard = get_answer_dashboard(context, question, examples_dashboard)
print(result_dashboard)


```html
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8" />
<meta name="viewport" content="width=device-width, initial-scale=1" />
<title>Expense Tracker Dashboard</title>
<script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
<style>
  :root {
    --bg-light: #f9f9f9;
    --text-light: #333;
    --card-light: #fff;
    --bg-dark: #121212;
    --text-dark: #eee;
    --card-dark: #1e1e1e;
    --primary: #2196f3;
    --success: #4caf50;
    --danger: #f44336;
  }
  body {
    margin: 0;
    font-family: Arial, sans-serif;
    background: var(--bg-light);
    color: var(--text-light);
    transition: all 0.3s ease;
  }
  body.dark {
    background: var(--bg-dark);
    color: var(--text-dark);
  }
  .container {
    max-width: 1000px;
    margin: auto;
    padding: 20px;
  }
  h1 {
    text-align: center;
    margin-bottom: 20px;
  }
  .summary {
    display: flex;
    flex-wrap: wrap;
    gap: 15px;
    margin-bottom: 20px;
  }
  .summary-card {
    flex: 1;
    min-wi

In [ ]:
# Contoh few-shot examples untuk Transactions Page
examples_transactions = [
    {
        "question": "Create a Transactions Management page for a budget tracker. Include a form to add transactions with date, description, category (income/expense), and amount. Display a table of transactions with edit and delete buttons.",
        "context": "Data stored in localStorage. Must support CRUD operations. Responsive design.",
        "output": """
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>Transactions</title>
<style>
/* Contoh CSS singkat */
body { font-family: Arial, sans-serif; padding: 20px; }
form, table { max-width: 600px; margin: auto; }
</style>
</head>
<body>
<h1>Transactions Management</h1>
<!-- Form -->
<form id="transaction-form">
  <input type="date" id="date" required>
  <input type="text" id="description" placeholder="Description" required>
  <select id="category" required>
    <option value="income">Income</option>
    <option value="expense">Expense</option>
  </select>
  <input type="number" id="amount" placeholder="Amount" required>
  <button type="submit">Add</button>
</form>

<!-- Table -->
<table id="transactions-table" border="1">
<thead>
<tr>
  <th>Date</th>
  <th>Description</th>
  <th>Category</th>
  <th>Amount</th>
  <th>Actions</th>
</tr>
</thead>
<tbody></tbody>
</table>

<script>
let transactions = JSON.parse(localStorage.getItem('transactions')) || [];

function renderTransactions() {
  const tbody = document.querySelector('#transactions-table tbody');
  tbody.innerHTML = '';
  transactions.forEach((t, index) => {
    tbody.innerHTML += \`
      <tr>
        <td>\${t.date}</td>
        <td>\${t.description}</td>
        <td>\${t.category}</td>
        <td>\${t.amount}</td>
        <td>
          <button onclick="editTransaction(\${index})">Edit</button>
          <button onclick="deleteTransaction(\${index})">Delete</button>
        </td>
      </tr>
    \`;
  });
}

document.querySelector('#transaction-form').addEventListener('submit', e => {
  e.preventDefault();
  const newTransaction = {
    date: date.value,
    description: description.value,
    category: category.value,
    amount: parseFloat(amount.value)
  };
  transactions.push(newTransaction);
  localStorage.setItem('transactions', JSON.stringify(transactions));
  renderTransactions();
  e.target.reset();
});

function deleteTransaction(index) {
  transactions.splice(index, 1);
  localStorage.setItem('transactions', JSON.stringify(transactions));
  renderTransactions();
}

function editTransaction(index) {
  const t = transactions[index];
  date.value = t.date;
  description.value = t.description;
  category.value = t.category;
  amount.value = t.amount;
  deleteTransaction(index);
}

renderTransactions();
</script>
</body>
</html>
"""
    }
]

def fewshot_prompt_transactions(context, question, examples):
    formatted_examples = "\n\n".join(
        f"""
Example {i+1}:
User Question: {ex['question']}
Context: {ex['context']}
Model Output:
{ex['output']}
        """ for i, ex in enumerate(examples)
    )

    prompt = f"""
You are an expert full-stack developer with 15 years of experience.
Your task is to generate a high-quality HTML, CSS, and JavaScript code for the Transactions Management page
of a Personal Expense Tracker app, based on the context and user question below.

Requirements:
- Must include complete HTML structure, internal or inline CSS, and JavaScript.
- Responsive design (mobile-friendly).
- CRUD operations (Create, Read, Update, Delete) for transactions.
- Save and retrieve data from localStorage.
- Automatically log changes to Activity History (via localStorage).
- Output only the complete code for this page. Do not include explanations outside of the code.

Here are examples of similar tasks you have completed before:
{formatted_examples}

Now generate the Transactions page code:

Context:
{context}

User Question:
{question}

Output:
"""
    return prompt

def get_answer_transactions(context, question, examples):
    prompt = fewshot_prompt_transactions(context, question, examples)
    result = model.invoke(prompt)
    return result

# Example usage
context = "Transactions page includes a form for adding transactions (date, description, category, amount), a table to display them, edit and delete functions. Data is stored in localStorage. Page is responsive. Logs all changes to Activity History."
question = "Create the Transactions page for the Personal Expense Tracker app. The page should allow adding, editing, deleting transactions with date, description, category (income/expense), and amount fields. Display them in a responsive table. Save to localStorage and log changes to Activity History."
result_transactions = get_answer_transactions(context, question, examples_transactions)
print(result_transactions)


```html
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>Transactions Management</title>
<style>
body { font-family: Arial, sans-serif; padding: 20px; }
form, table { max-width: 100%; margin: auto; }
th, td { padding: 10px; text-align: left; border-bottom: 1px solid #ddd; }
@media (max-width: 600px) {
  form, table { width: 95%; }
}
</style>
</head>
<body>
<h1>Transactions Management</h1>

<!-- Form -->
<form id="transaction-form">
  <input type="date" id="date" required>
  <input type="text" id="description" placeholder="Description" required>
  <select id="category" required>
    <option value="income">Income</option>
    <option value="expense">Expense</option>
  </select>
  <input type="number" id="amount" placeholder="Amount" required>
  <button type="submit">Add</button>
</form>

<!-- Table -->
<table id="transactions-table" border="1">
<thead>
<tr>
  <th>Date</th>
  <th>Description</th>
 

In [ ]:
# Few-shot examples untuk Activity History Page
examples_history = [
    {
        "question": "Create an Activity History page for a personal finance app. Display logs of actions like add/edit/delete transactions with timestamps. Include filters and clear log button.",
        "context": "Data stored in localStorage under 'activityLog'. Page is responsive and mobile-friendly.",
        "output": """
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>Activity History</title>
<style>
body { font-family: Arial, sans-serif; padding: 20px; max-width: 800px; margin: auto; }
h1 { text-align: center; }
table { width: 100%; border-collapse: collapse; margin-top: 20px; }
th, td { border: 1px solid #ccc; padding: 8px; text-align: left; }
th { background-color: #f4f4f4; }
.controls { display: flex; gap: 10px; margin-top: 10px; flex-wrap: wrap; }
button { padding: 5px 10px; }
</style>
</head>
<body>

<h1>Activity History</h1>

<div class="controls">
  <select id="filter-action">
    <option value="">All Actions</option>
    <option value="add">Add</option>
    <option value="edit">Edit</option>
    <option value="delete">Delete</option>
  </select>
  <input type="date" id="filter-date">
  <button onclick="clearLog()">Clear All History</button>
</div>

<table id="history-table">
<thead>
<tr>
  <th>Timestamp</th>
  <th>Action</th>
  <th>Details</th>
</tr>
</thead>
<tbody></tbody>
</table>

<script>
let activityLog = JSON.parse(localStorage.getItem('activityLog')) || [];

function renderHistory() {
  const filterAction = document.getElementById('filter-action').value;
  const filterDate = document.getElementById('filter-date').value;
  const tbody = document.querySelector('#history-table tbody');
  tbody.innerHTML = '';

  activityLog
    .filter(entry => !filterAction || entry.action === filterAction)
    .filter(entry => !filterDate || entry.timestamp.startsWith(filterDate))
    .forEach(entry => {
      tbody.innerHTML += \`
        <tr>
          <td>\${entry.timestamp}</td>
          <td>\${entry.action}</td>
          <td>\${entry.details}</td>
        </tr>
      \`;
    });
}

function clearLog() {
  if (confirm("Are you sure you want to clear all history?")) {
    localStorage.removeItem('activityLog');
    activityLog = [];
    renderHistory();
  }
}

document.getElementById('filter-action').addEventListener('change', renderHistory);
document.getElementById('filter-date').addEventListener('change', renderHistory);

renderHistory();
</script>

</body>
</html>
"""
    }
]

def fewshot_prompt_history(context, question, examples):
    formatted_examples = "\n\n".join(
        f"""
Example {i+1}:
User Question: {ex['question']}
Context: {ex['context']}
Model Output:
{ex['output']}
        """ for i, ex in enumerate(examples)
    )

    prompt = f"""
You are an expert full-stack developer with 15 years of experience.
Your task is to generate a high-quality HTML, CSS, and JavaScript code for the Activity History page
of a Personal Expense Tracker app, based on the context and user question below.

Requirements:
- Must include complete HTML structure, internal or inline CSS, and JavaScript.
- Responsive design (mobile-friendly).
- Display logs of actions (add, edit, delete transactions) with timestamps and details.
- Allow filtering by action type and date.
- Include "Clear All History" button with confirmation.
- Data is stored in localStorage under 'activityLog'.
- Output only the complete code for this page. Do not include explanations outside of the code.

Here are examples of similar tasks you have completed before:
{formatted_examples}

Now generate the Activity History page code:

Context:
{context}

User Question:
{question}

Output:
"""
    return prompt

def get_answer_history(context, question, examples):
    prompt = fewshot_prompt_history(context, question, examples)
    result = model.invoke(prompt)
    return result

# Example usage
context = "Activity History page lists all user actions (add, edit, delete) on transactions with timestamp and details. Includes filters by action type and date, and a clear history button. Data stored in localStorage."
question = "Create the Activity History page for the Personal Expense Tracker app. Display a table of activity logs with timestamp, action type, and details. Include filter by action type and date, and a clear all button with confirmation."
result_history = get_answer_history(context, question, examples_history)
print(result_history)


<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>Activity History</title>
<style>
body { font-family: Arial, sans-serif; padding: 20px; max-width: 800px; margin: auto; }
h1 { text-align: center; }
table { width: 100%; border-collapse: collapse; margin-top: 20px; }
th, td { border: 1px solid #ccc; padding: 8px; text-align: left; }
th { background-color: #f4f4f4; }
.controls { display: flex; gap: 10px; margin-top: 10px; flex-wrap: wrap; }
button { padding: 5px 10px; }
@media (max-width: 600px) {
  table, .controls { width: 100%; }
}
</style>
</head>
<body>

<h1>Activity History</h1>

<div class="controls">
  <select id="filter-action">
    <option value="">All Actions</option>
    <option value="add">Add</option>
    <option value="edit">Edit</option>
    <option value="delete">Delete</option>
  </select>
  <input type="date" id="filter-date">
  <button onclick="clearLog()">Clear All History</butt

In [ ]:
# Few-shot examples untuk Analytics Page
examples_analytics = [
    {
        "question": "Create an Analytics page for a personal expense tracker app. Show Donut Chart for expense percentage by category with time range filters (weekly, monthly, yearly) and a simple insight text.",
        "context": "Data stored in localStorage under 'transactions'. Use Chart.js for charts. Responsive design required.",
        "output": """
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>Analytics</title>
<script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
<style>
body { font-family: Arial, sans-serif; padding: 20px; max-width: 900px; margin: auto; }
h1 { text-align: center; }
.controls { display: flex; gap: 10px; margin-bottom: 20px; flex-wrap: wrap; }
canvas { max-width: 100%; }
#insight { margin-top: 20px; font-weight: bold; }
</style>
</head>
<body>

<h1>Expense Analytics</h1>

<div class="controls">
  <select id="time-filter">
    <option value="all">All Time</option>
    <option value="weekly">This Week</option>
    <option value="monthly">This Month</option>
    <option value="yearly">This Year</option>
  </select>
</div>

<canvas id="donutChart"></canvas>
<div id="insight"></div>

<script>
let transactions = JSON.parse(localStorage.getItem('transactions')) || [];

function filterByTime(period) {
  const now = new Date();
  return transactions.filter(t => {
    const date = new Date(t.date);
    if (period === 'weekly') {
      const oneWeekAgo = new Date();
      oneWeekAgo.setDate(now.getDate() - 7);
      return date >= oneWeekAgo;
    } else if (period === 'monthly') {
      return date.getMonth() === now.getMonth() && date.getFullYear() === now.getFullYear();
    } else if (period === 'yearly') {
      return date.getFullYear() === now.getFullYear();
    }
    return true;
  });
}

function updateChart() {
  const period = document.getElementById('time-filter').value;
  const filtered = filterByTime(period);

  const expenses = filtered.filter(t => t.type === 'expense');
  const categories = [...new Set(expenses.map(e => e.category))];
  const categoryTotals = categories.map(cat =>
    expenses.filter(e => e.category === cat)
            .reduce((sum, e) => sum + e.amount, 0)
  );

  donutChart.data.labels = categories;
  donutChart.data.datasets[0].data = categoryTotals;
  donutChart.update();

  if (categoryTotals.length > 0) {
    const maxIndex = categoryTotals.indexOf(Math.max(...categoryTotals));
    document.getElementById('insight').innerText =
      \`Biggest expense category: \${categories[maxIndex]} (\${categoryTotals[maxIndex].toFixed(2)})\`;
  } else {
    document.getElementById('insight').innerText = "No expenses recorded for this period.";
  }
}

const ctx = document.getElementById('donutChart').getContext('2d');
const donutChart = new Chart(ctx, {
  type: 'doughnut',
  data: {
    labels: [],
    datasets: [{
      label: 'Expense Percentage by Category',
      data: [],
      backgroundColor: ['#FF6384', '#36A2EB', '#FFCE56', '#4BC0C0', '#9966FF', '#FF9F40']
    }]
  },
  options: { responsive: true }
});

document.getElementById('time-filter').addEventListener('change', updateChart);

updateChart();
</script>

</body>
</html>
"""
    }
]

def fewshot_prompt_analytics(context, question, examples):
    formatted_examples = "\n\n".join(
        f"""
Example {i+1}:
User Question: {ex['question']}
Context: {ex['context']}
Model Output:
{ex['output']}
        """ for i, ex in enumerate(examples)
    )

    prompt = f"""
You are an expert full-stack developer with 15 years of experience.
Your task is to generate a high-quality HTML, CSS, and JavaScript code for the Analytics page
of a Personal Expense Tracker app, based on the context and user question below.

Requirements:
- Must include complete HTML, CSS, and JavaScript in one file.
- Show Donut Chart (expense percentage by category) using Chart.js.
- Include a time range filter (weekly, monthly, yearly, all).
- Generate an insight text showing the largest expense category for the selected period.
- Responsive and mobile-friendly.
- Data is stored in localStorage under 'transactions'.
- Output only the complete code for this page. No extra explanations.

Here are examples of similar tasks you have completed before:
{formatted_examples}

Now generate the Analytics page code:

Context:
{context}

User Question:
{question}

Output:
"""
    return prompt

def get_answer_analytics(context, question, examples):
    prompt = fewshot_prompt_analytics(context, question, examples)
    result = model.invoke(prompt)
    return result

# Example usage
context = "Analytics page with Donut Chart showing expense percentage by category. Includes time filters (weekly, monthly, yearly) and insight text. Data stored in localStorage."
question = "Create the Analytics page for the Personal Expense Tracker app with a Donut Chart showing expense percentage by category, a time filter dropdown, and insight text showing the largest category."
result_analytics = get_answer_analytics(context, question, examples_analytics)
print(result_analytics)


<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>Expense Analytics</title>
<script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
<style>
body { font-family: Arial, sans-serif; padding: 20px; max-width: 900px; margin: auto; }
h1 { text-align: center; }
.controls { display: flex; gap: 10px; margin-bottom: 20px; flex-wrap: wrap; }
canvas { max-width: 100%; }
#insight { margin-top: 20px; font-weight: bold; }
</style>
</head>
<body>

<h1>Expense Analytics</h1>

<div class="controls">
  <select id="time-filter">
    <option value="all">All Time</option>
    <option value="weekly">This Week</option>
    <option value="monthly">This Month</option>
    <option value="yearly">This Year</option>
  </select>
</div>

<canvas id="donutChart"></canvas>
<div id="insight"></div>

<script>
let transactions = JSON.parse(localStorage.getItem('transactions')) || [];

function filterByTime(period) {
  const n